# GoProtect_2

#### Цель работы

* Создать модель машинного обучения для сопоставления названий спортивных школ, записанных в разном формате (в том числе и с ошибками) с эталоном.


#### Ход работы
* Загрузить и изучить данные
* Создать обучающий датасет
* Подготовить решение по мэтчингу произвольных вариантов написания названий и эталонных
* Дообучить модель
* Выполнить тестирование и сравнить результаты исходной модели и дообученной
* Сделать описание решения, выводы и рекомендации

### исходные данные
* "Примерное написание.csv" - набор вариантов для тестирования
* "Школы.csv" - эталонные названия школ и регионы


In [1]:
import pandas as pd
import numpy as np
import re


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.svm import LinearSVC

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer



from sentence_transformers import SentenceTransformer, InputExample, losses, SentenceTransformerTrainer
from sentence_transformers.util import semantic_search
import spacy
import nlpaug
from nltk.corpus import stopwords
import nlpaug.augmenter.char as nac

from torch.utils.data import DataLoader, Dataset

import tensorflow as tf
import keras

import warnings

c:\Users\Freo\Desktop\kaggle\protect3\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
nlp = spacy.load("ru_core_news_lg")

In [4]:
RSTATE = 12345

In [5]:
df_for_test = pd.read_csv('C:\\Users\\Freo\\Desktop\\kaggle\\protect\\Примерное написание.csv')

In [6]:
df_for_test.head()

,school_id,name
0,1836,"ООО ""Триумф"""
1,1836,"Москва, СК ""Триумф"""
2,610,"СШОР ""Надежда Губернии"
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
4,609,"""СШ ""Гвоздика"""


In [7]:
df = pd.read_csv('C:\\Users\\Freo\\Desktop\\kaggle\\protect\\Школы.csv')

In [8]:
df.head()

,school_id,name,region
0,1,Авангард,Московская область
1,2,Авангард,Ямало-Ненецкий АО
2,3,Авиатор,Республика Татарстан
3,4,Аврора,Санкт-Петербург
4,5,Ice Dream / Айс Дрим,Санкт-Петербург


In [9]:
df

,school_id,name,region
0,1,Авангард,Московская область
1,2,Авангард,Ямало-Ненецкий АО
2,3,Авиатор,Республика Татарстан
3,4,Аврора,Санкт-Петербург
4,5,Ice Dream / Айс Дрим,Санкт-Петербург
...,...,...,...
301,305,Прогресс,Алтайский край
302,609,"""СШ ""Гвоздика""",Удмуртская республика
303,610,"СШОР ""Надежда Губернии",Саратовская область
304,611,КФК «Айсберг»,Пермский край


In [10]:
df['name'] = df['region'] + ' ' + df['name']

In [11]:
df = df.drop(columns='region')

In [12]:
df['name_orig'] = df['name'] 

In [13]:
df

,school_id,name,name_orig
0,1,Московская область Авангард,Московская область Авангард
1,2,Ямало-Ненецкий АО Авангард,Ямало-Ненецкий АО Авангард
2,3,Республика Татарстан Авиатор,Республика Татарстан Авиатор
3,4,Санкт-Петербург Аврора,Санкт-Петербург Аврора
4,5,Санкт-Петербург Ice Dream / Айс Дрим,Санкт-Петербург Ice Dream / Айс Дрим
...,...,...,...
301,305,Алтайский край Прогресс,Алтайский край Прогресс
302,609,"Удмуртская республика ""СШ ""Гвоздика""","Удмуртская республика ""СШ ""Гвоздика"""
303,610,"Саратовская область СШОР ""Надежда Губернии","Саратовская область СШОР ""Надежда Губернии"
304,611,Пермский край КФК «Айсберг»,Пермский край КФК «Айсберг»


In [14]:
def lemmatize(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

* создадим обучающие признаки из названий школ
* создадим целевые признаки из уникальных id школ

In [15]:
x_train = df['name']
y_train_final = df['school_id']

In [16]:
x_train.head()

0             Московская область Авангард
1              Ямало-Ненецкий АО Авангард
2            Республика Татарстан Авиатор
3                  Санкт-Петербург Аврора
4    Санкт-Петербург Ice Dream / Айс Дрим
Name: name, dtype: object

In [17]:
df_for_test.head()

,school_id,name
0,1836,"ООО ""Триумф"""
1,1836,"Москва, СК ""Триумф"""
2,610,"СШОР ""Надежда Губернии"
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
4,609,"""СШ ""Гвоздика"""


Функция для очистки текста от лишних символов и пунктуации

In [18]:
def clear_text(text):
    text = re.sub(r'[^A-Za-zА-Яа-яЁё1-90]', ' ', text)
    text = re.sub(r',\"', ' ', text)
    text = text.split()
    res = ' '.join(text)
    # return res.lower()
    return res

обработаем признаки из итоговой проверяющей выборки под общий стандарт очистки текста

In [19]:
x_test_final = df_for_test['name'].apply(clear_text)
y_test_final = df_for_test['school_id']

создадим корпуса текста для векторизации с помощью tf idf

In [20]:
corpus_x_train = x_train.values
corpus_x_test_final = x_test_final.values

In [21]:
count_tf_idf = TfidfVectorizer(ngram_range=(1,1))

векторизируем

In [22]:
tf_idf_train = count_tf_idf.fit_transform(corpus_x_train)
tf_idf_test_final = count_tf_idf.transform(corpus_x_test_final)

рассмотрим бейслайн модель, обучим ее на имеющихся данных для понимания точки отсчета и сложности задачи

In [23]:
model = LogisticRegression(solver='liblinear', random_state=RSTATE, class_weight = 'balanced')
model.fit(tf_idf_train,y_train_final)
pred = model.predict(tf_idf_test_final)

print('acc',accuracy_score(pred,y_test_final))

acc 0.841340782122905


для такой небольшой выборки бейслайн модель показала отличный результат, однако:
* выборка исчезающе мала, при появлении новых данных (в том числе ошибок и опечаток) ее качество прогнозов может упасть очень сильно


### расширение тренировочной выборки. Обычные МЛ модели.

С помощью библиотеки nlpaug создадим новые записи, моделирующие разные ошибки и опечатки при заполнении:
* замена случайной буквы в тексте на случайную соседнюю на клавиатуре
* удаление случайной буквы в тексте
* замена двух находящихся рядом букв

подобный подход позволит создать очень реалистичные записи на основе имеющихся данных

В библиотеке нет готовой реализации для работы с русским языком, однако есть функционал для персонализации.
С учетом необходимых для проекта задач можно немного срезать угол, и вместо заполнения полного словаря символов, достаточно выбрать в аргументах украинский язык, и заменить 2 отличающиеся буквы в раскладке клавиатуры.

Функция замены для двух отличающихся букв

In [24]:
def to_rus(text):
    text = re.sub(r'[iIІі]', 'ы', text)
    text = re.sub(r'[Її]', 'ъ', text)
    return text

создадим разные варианты настроек для нескольких типов ошибок. В завимости от особенностей реального, рабочего датасета можно будет изменять параметры, подбирая те, с которыми моделированный датасет будет наиболее точно отражать реальную картину

In [25]:
aug2 = nac.KeyboardAug(lang='uk',aug_char_min=1, aug_char_max=1, aug_char_p=0.1, aug_word_p=0.1)
aug3 = nac.KeyboardAug(lang='uk',aug_char_min=1, aug_char_max=2, aug_char_p=0.2, aug_word_p=0.1)
aug4 = nac.KeyboardAug(lang='uk',aug_char_min=1, aug_char_max=3, aug_char_p=0.3, aug_word_p=0.1)
aug5 = nac.RandomCharAug(action="swap", aug_char_min=1, aug_char_max=1, aug_char_p=0.1, aug_word_p=0.1)
aug6 = nac.RandomCharAug(action="swap", aug_char_min=2, aug_char_max=2, aug_char_p=0.2, aug_word_p=0.2)
aug7 = nac.RandomCharAug(action="delete", aug_char_min=1, aug_char_max=1, aug_char_p=0.1, aug_word_p=0.1)
aug8 = nac.RandomCharAug(action="delete", aug_char_min=2, aug_char_max=2, aug_char_p=0.2, aug_word_p=0.2)

создадим шаблон

In [26]:
df2 = df[['name','name_orig','school_id']]
df3 = df[['name','name_orig','school_id']]
df4 = df[['name','name_orig','school_id']]
df5 = df[['name','name_orig','school_id']]
df6 = df[['name','name_orig','school_id']]
df7 = df[['name','name_orig','school_id']]
df8 = df[['name','name_orig','school_id']]

создадим аугментированные записи на основе реальных

In [27]:
df2['name'] = df['name'].apply(lambda x: aug2.augment(x)[0]).apply(to_rus)
df3['name'] = df['name'].apply(lambda x: aug3.augment(x)[0]).apply(to_rus)
df4['name'] = df['name'].apply(lambda x: aug4.augment(x)[0]).apply(to_rus)
df5['name'] = df['name'].apply(lambda x: aug5.augment(x)[0]).apply(to_rus)
df6['name'] = df['name'].apply(lambda x: aug6.augment(x)[0]).apply(to_rus)
df7['name'] = df['name'].apply(lambda x: aug7.augment(x)[0]).apply(to_rus)
df8['name'] = df['name'].apply(lambda x: aug8.augment(x)[0]).apply(to_rus)

выберем необходимые кусочки для собирания датасета

In [28]:
df_big = pd.concat([df, df2, df5, df7], ignore_index=True)
# df_big = pd.concat([df, df2, df3, df5, df7], ignore_index=True)
# df_big = pd.concat([df, df2, df3, df4, df5, df6, df7, df8], ignore_index=True)

применим функцию очистки к получившемуся датасету, для удаления ненужных знаков препринания и стандартизации данных

In [29]:
df_big['name'] = df_big['name'].apply(clear_text)


In [30]:
df_big.head()

,school_id,name,name_orig
0,1,Московская область Авангард,Московская область Авангард
1,2,Ямало Ненецкий АО Авангард,Ямало-Ненецкий АО Авангард
2,3,Республика Татарстан Авиатор,Республика Татарстан Авиатор
3,4,Санкт Петербург Аврора,Санкт-Петербург Аврора
4,5,Санкт Петербург Ice Dream Айс Дрим,Санкт-Петербург Ice Dream / Айс Дрим


выделим целевой и обучающие признаки из нового смоделированного датасета

In [31]:
x = df_big['name']
y = df_big['school_id']

разделим на выборки

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=RSTATE)

In [33]:
x_test_final

0                                           ООО Триумф
1                                     Москва СК Триумф
2                                СШОР Надежда Губернии
3      Саратовская область ГБУСО СШОР Надежда Губернии
4                                          СШ Гвоздика
                            ...                       
890              Республика Татарстан СШОР ФСО Авиатор
891              СШОР ФСО Авиатор Республика Татарстан
892       Республика Татарстан МБУ ДО СШОР ФСО Авиатор
893                                   ЯНАО СШ Авангард
894                     Московская область СШ Авангард
Name: name, Length: 895, dtype: object

создание корпусов текста и векторизация

In [34]:
corpus_x_train = x_train.values
corpus_x_test = x_test.values
corpus_x_test_final = x_test_final.values

In [35]:
count_tf_idf = TfidfVectorizer(ngram_range=(1,1))

In [36]:
tf_idf_train = count_tf_idf.fit_transform(corpus_x_train)
tf_idf_test = count_tf_idf.transform(corpus_x_test)
tf_idf_test_final = count_tf_idf.transform(corpus_x_test_final)


## Модели

посмотрим как покажут себя модели на смоделированном датасете, выберем лучшую модель и проверим ее:
* на тестовых данных
* на итоговой проверочной выборке

### Логистическая регрессия

In [37]:
model = LogisticRegression(random_state=RSTATE)

param = {
    'solver': ['liblinear'],
    'C': range (10, 500, 10),
}


gs_logreg = GridSearchCV(model, param, cv=3, scoring='accuracy')
gs_logreg.fit(tf_idf_train,y_train)


print('Best parameters: ', gs_logreg.best_params_)
print('Best accuracy: ', gs_logreg.best_score_)

Best parameters:  {'C': 110, 'solver': 'liblinear'}
Best accuracy:  0.7977430066978105


### Решающее дерево

In [38]:
model = DecisionTreeClassifier(random_state=RSTATE)

param = {
    'max_depth': range(100,210,1),
    'splitter': ['best','random'],
}


gs_dt = GridSearchCV(model, param, cv=3, scoring='accuracy')
gs_dt.fit(tf_idf_train,y_train)


print('Best parameters: ', gs_dt.best_params_)
print('Best accuracy: ', gs_dt.best_score_)

Best parameters:  {'max_depth': 152, 'splitter': 'random'}
Best accuracy:  0.593428516037848


### Случайный лес

In [39]:
model = RandomForestClassifier(random_state=RSTATE)

param = {
    'max_depth': range(100,155,10),
    'n_estimators':range(1, 151, 10)
}


gs_rf = GridSearchCV(model, param, cv=3, scoring='accuracy')
gs_rf.fit(tf_idf_train,y_train)


print('Best parameters: ', gs_rf.best_params_)
print('Best accuracy: ', gs_rf.best_score_)

Best parameters:  {'max_depth': 150, 'n_estimators': 131}
Best accuracy:  0.7252584379279939


Бустинги с таким набором данных работают невероятно долго, использовать их как основную модель для решения подобной задачи на мой взгляд нерационально, однако существуют варианты рассмотреть их в комбинациях с другими моделями.

На этом этапе работы с синтезированным датасетом лучше всех показала себя модель линейной регрессии. Проверим ее на тестовой выборке синтезированного датасета, а так же на выборке для проверки от заказчика

### Линейная - синтезированный датасет - тестовая выборка

In [40]:
model = LogisticRegression(solver='liblinear', penalty='l2', C=20, random_state=RSTATE, class_weight = 'balanced')
model.fit(tf_idf_train,y_train)

pred = model.predict(tf_idf_test)
print('acc',accuracy_score(pred,y_test))

acc 0.7673469387755102


### Линейная - проверяющий датасет

In [41]:
model = LogisticRegression(solver='liblinear', penalty='l2', C=20, random_state=RSTATE, class_weight = 'balanced')
model.fit(tf_idf_train,y_train)

pred = model.predict(tf_idf_test_final)
print('acc',accuracy_score(pred,y_test_final))

acc 0.7675977653631285


Падение метрики обусловлено отличием имеющихся данных от возможных ошибок; модель стала немного хуже определять размеченные данные, но гораздно лучше - данные со случайными ошибками

## Расширение тренировочной выборки. BERT sentence embeddings

повторим логику создания нового большого датафрейма, с небольшими отличиями в заполнении:

для языковой модели LaBSE создадим признаки аугментированных названий, чтобы сравнить их с названиями из эталонного датасета

In [42]:
df2 = df[['name','name_orig','school_id']]
df3 = df[['name','name_orig','school_id']]
df4 = df[['name','name_orig','school_id']]
df5 = df[['name','name_orig','school_id']]
df6 = df[['name','name_orig','school_id']]
df7 = df[['name','name_orig','school_id']]
df8 = df[['name','name_orig','school_id']]

если будет необходимость отдельно настроить параметры

In [43]:
# aug2 = nac.KeyboardAug(lang='uk',aug_char_min=1, aug_char_max=1, aug_char_p=0.1, aug_word_p=0.1)
# aug3 = nac.KeyboardAug(lang='uk',aug_char_min=1, aug_char_max=2, aug_char_p=0.2, aug_word_p=0.1)
# aug4 = nac.KeyboardAug(lang='uk',aug_char_min=1, aug_char_max=3, aug_char_p=0.3, aug_word_p=0.1)
# aug5 = nac.RandomCharAug(action="swap", aug_char_min=1, aug_char_max=1, aug_char_p=0.1, aug_word_p=0.1)
# aug6 = nac.RandomCharAug(action="swap", aug_char_min=2, aug_char_max=2, aug_char_p=0.2, aug_word_p=0.2)
# aug7 = nac.RandomCharAug(action="delete", aug_char_min=1, aug_char_max=1, aug_char_p=0.1, aug_word_p=0.1)
# aug8 = nac.RandomCharAug(action="delete", aug_char_min=2, aug_char_max=2, aug_char_p=0.2, aug_word_p=0.2)

In [44]:
df2['name_aug'] = df['name'].apply(lambda x: aug2.augment(x)[0]).apply(to_rus)
df3['name_aug'] = df['name'].apply(lambda x: aug3.augment(x)[0]).apply(to_rus)
df4['name_aug'] = df['name'].apply(lambda x: aug4.augment(x)[0]).apply(to_rus)
df5['name_aug'] = df['name'].apply(lambda x: aug5.augment(x)[0]).apply(to_rus)
df6['name_aug'] = df['name'].apply(lambda x: aug6.augment(x)[0]).apply(to_rus)
df7['name_aug'] = df['name'].apply(lambda x: aug7.augment(x)[0]).apply(to_rus)
df8['name_aug'] = df['name'].apply(lambda x: aug8.augment(x)[0]).apply(to_rus)

создадим большой датасет со всеми видами ошибок и разными вероятностями их появления

In [45]:
# df_big = pd.concat([df2, df5, df7], ignore_index=True)
# df_big = pd.concat([df2, df3, df5, df7], ignore_index=True)
df_big = pd.concat([df2,df3, df4, df5,df6, df7, df8], ignore_index=True)

In [46]:
df_big.head()

,name,name_orig,school_id,name_aug
0,Московская область Авангард,Московская область Авангард,1,Москлвская область Авангард
1,Ямало-Ненецкий АО Авангард,Ямало-Ненецкий АО Авангард,2,фмало - Ненецкий АО Авангард
2,Республика Татарстан Авиатор,Республика Татарстан Авиатор,3,Республика Татарытан Авиатор
3,Санкт-Петербург Аврора,Санкт-Петербург Аврора,4,Санкт - Петербург Асрора
4,Санкт-Петербург Ice Dream / Айс Дрим,Санкт-Петербург Ice Dream / Айс Дрим,5,Санкт - Ретербург ыce Dream / Айс Дрим


стандартизируем вид аугментированной и оригинальной колонки имен: избавимся от лишней пунктуации пунктуации

In [47]:
df_big['name'] = df_big['name'].apply(clear_text)

In [48]:
df_big['name_aug'] = df_big['name_aug'].apply(clear_text)

In [49]:
df_big.head()

,name,name_orig,school_id,name_aug
0,Московская область Авангард,Московская область Авангард,1,Москлвская область Авангард
1,Ямало Ненецкий АО Авангард,Ямало-Ненецкий АО Авангард,2,фмало Ненецкий АО Авангард
2,Республика Татарстан Авиатор,Республика Татарстан Авиатор,3,Республика Татарытан Авиатор
3,Санкт Петербург Аврора,Санкт-Петербург Аврора,4,Санкт Петербург Асрора
4,Санкт Петербург Ice Dream Айс Дрим,Санкт-Петербург Ice Dream / Айс Дрим,5,Санкт Ретербург ыce Dream Айс Дрим


разделим получившийся датасет на две части:

* создадим модель обучив на небольшой выборке
* дообучим на оставшейся части
* сравним результаты до и после дообучения
* сравним результаты с моделью обученном сразу на полном датасете

In [50]:
big_train, min_train = train_test_split(df_big, test_size=0.2, random_state=RSTATE)

### sentence-transformers/LaBSE

In [51]:
model = SentenceTransformer('sentence-transformers/LaBSE')

In [52]:
corpus = model.encode(df['name'].values)
query = model.encode(min_train['name_aug'].values)

In [53]:
search_result = semantic_search(query,corpus, top_k=5)

In [54]:
min_train['candidate_indxs'] = [[x['corpus_id'] for x in row] for row in search_result]

In [56]:
min_train['candidate_names'] = min_train['candidate_indxs'].transform(lambda x: df['name_orig'].values[x])

In [57]:
min_train['candidate_school_id'] = min_train['candidate_indxs'].transform(lambda x: df['school_id'].values[x])

In [58]:
min_train.head()

,name,name_orig,school_id,name_aug,candidate_indxs,candidate_names,candidate_school_id
895,Санкт Петербург СК ФК Ice Prestige,Санкт-Петербург СК ФК «Ice Prestige»,286,ыанПт Петербург СК ФК ыce Prestыge,"[283, 255, 104, 267, 254]","[Санкт-Петербург СК ФК «Ice Prestige», Санкт-П...","[286, 257, 105, 270, 256]"
33,Москва Волкова,Москва Волкова,34,Москва Вллкова,"[33, 137, 170, 100, 243]","[Москва Волкова, Москва Прусова, Москва Столиц...","[34, 138, 171, 101, 244]"
436,Санкт Петербург Пируэт,Санкт-Петербург Пируэт,131,Санкт Нетербуег Пируэт,"[130, 26, 133, 273, 110]","[Санкт-Петербург Пируэт, Санкт-Петербург Бурев...","[131, 27, 134, 276, 111]"
521,Ростовская область СШОР 6,Ростовская область СШОР № 6,216,Ростовская область СШБР 6,"[215, 216, 140, 154, 173]","[Ростовская область СШОР № 6, Рязанская област...","[216, 217, 141, 155, 174]"
1294,Калининградская область КО СШ по ЗВС,Калининградская область КО СШ по ЗВС,71,Каилнинградскяа олбатсь КО СШ по ЗВС,"[70, 178, 198, 173, 175]","[Калининградская область КО СШ по ЗВС, Краснод...","[71, 179, 199, 174, 176]"


In [59]:
min_train.head(2)

,name,name_orig,school_id,name_aug,candidate_indxs,candidate_names,candidate_school_id
895,Санкт Петербург СК ФК Ice Prestige,Санкт-Петербург СК ФК «Ice Prestige»,286,ыанПт Петербург СК ФК ыce Prestыge,"[283, 255, 104, 267, 254]","[Санкт-Петербург СК ФК «Ice Prestige», Санкт-П...","[286, 257, 105, 270, 256]"
33,Москва Волкова,Москва Волкова,34,Москва Вллкова,"[33, 137, 170, 100, 243]","[Москва Волкова, Москва Прусова, Москва Столиц...","[34, 138, 171, 101, 244]"


In [60]:
min_train['top1'] = min_train.apply(lambda x: x['school_id'] == x['candidate_school_id'][0], axis=1)
min_train['top5'] = min_train.apply(lambda x: x['school_id'] in x['candidate_school_id'], axis=1)

In [61]:
min_train.head(2)

,name,name_orig,school_id,name_aug,candidate_indxs,candidate_names,candidate_school_id,top1,top5
895,Санкт Петербург СК ФК Ice Prestige,Санкт-Петербург СК ФК «Ice Prestige»,286,ыанПт Петербург СК ФК ыce Prestыge,"[283, 255, 104, 267, 254]","[Санкт-Петербург СК ФК «Ice Prestige», Санкт-П...","[286, 257, 105, 270, 256]",True,True
33,Москва Волкова,Москва Волкова,34,Москва Вллкова,"[33, 137, 170, 100, 243]","[Москва Волкова, Москва Прусова, Москва Столиц...","[34, 138, 171, 101, 244]",True,True


In [62]:
min_train['top1'].sum() / df_big.shape[0]

0.19327731092436976

In [63]:
min_train['top5'].sum() / df_big.shape[0]

0.20028011204481794

модель показывает весьма скромные результаты, всего 20% точности определения школы, однако и обучена она всего на 20% моделированного датасета

дообучим модель оставшейся частью датасета:

In [64]:
make_example = lambda x: InputExample(texts=[x['name'], x['name_aug']])
examples = big_train[['name','name_aug']].apply(make_example, axis=1).values
train_dataloader = DataLoader(examples, shuffle=True, batch_size=16)
train_loss = losses.GISTEmbedLoss(model=model, guide=model)
# train_loss = losses.MultipleNegativesRankingLoss(model = model)
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2)

  0%|          | 0/216 [00:00<?, ?it/s]

{'train_runtime': 875.1288, 'train_samples_per_second': 3.915, 'train_steps_per_second': 0.247, 'train_loss': 0.08689246354279695, 'epoch': 2.0}


### sentence-transformers/LaBSE - проверяющий датасет

проверим модель на проверяющем датасете:

In [65]:
x_test_final

0                                           ООО Триумф
1                                     Москва СК Триумф
2                                СШОР Надежда Губернии
3      Саратовская область ГБУСО СШОР Надежда Губернии
4                                          СШ Гвоздика
                            ...                       
890              Республика Татарстан СШОР ФСО Авиатор
891              СШОР ФСО Авиатор Республика Татарстан
892       Республика Татарстан МБУ ДО СШОР ФСО Авиатор
893                                   ЯНАО СШ Авангард
894                     Московская область СШ Авангард
Name: name, Length: 895, dtype: object

In [66]:
query = model.encode(x_test_final.values)
search_result = semantic_search(query,corpus, top_k=5)

df_for_test['candidate_indxs'] = [[x['corpus_id'] for x in row] for row in search_result]
df_for_test['candidate_names'] = df_for_test['candidate_indxs'].transform(lambda x: df['name_orig'].values[x])
df_for_test['candidate_school_id'] = df_for_test['candidate_indxs'].transform(lambda x: df['school_id'].values[x])

df_for_test['top1'] = df_for_test.apply(lambda x: x['school_id'] == x['candidate_school_id'][0], axis=1)
df_for_test['top5'] = df_for_test.apply(lambda x: x['school_id'] in x['candidate_school_id'], axis=1)

df_for_test.head()

,school_id,name,candidate_indxs,candidate_names,candidate_school_id,top1,top5
0,1836,"ООО ""Триумф""","[305, 224, 299, 286, 134]","[Москва ООО ""Триумф"", Пермский край Триумф, Мо...","[1836, 225, 303, 289, 135]",True,True
1,1836,"Москва, СК ""Триумф""","[305, 95, 250, 251, 104]","[Москва ООО ""Триумф"", Москва МАФКК, Москва ЦСК...","[1836, 96, 251, 252, 105]",True,True
2,610,"СШОР ""Надежда Губернии","[303, 302, 23, 36, 182]","[Саратовская область СШОР ""Надежда Губернии, У...","[610, 609, 24, 37, 183]",True,True
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе...","[303, 36, 71, 193, 173]","[Саратовская область СШОР ""Надежда Губернии, Н...","[610, 37, 72, 194, 174]",True,True
4,609,"""СШ ""Гвоздика""","[302, 172, 180, 184, 160]","[Удмуртская республика ""СШ ""Гвоздика"", ХМАО-Юг...","[609, 173, 181, 185, 161]",True,True


In [67]:
df_for_test['top1'].sum() / df_for_test.shape[0]

0.6983240223463687

In [68]:
df_for_test['top5'].sum() / df_for_test.shape[0]

0.9217877094972067

In [69]:
df_for_test.head()

,school_id,name,candidate_indxs,candidate_names,candidate_school_id,top1,top5
0,1836,"ООО ""Триумф""","[305, 224, 299, 286, 134]","[Москва ООО ""Триумф"", Пермский край Триумф, Мо...","[1836, 225, 303, 289, 135]",True,True
1,1836,"Москва, СК ""Триумф""","[305, 95, 250, 251, 104]","[Москва ООО ""Триумф"", Москва МАФКК, Москва ЦСК...","[1836, 96, 251, 252, 105]",True,True
2,610,"СШОР ""Надежда Губернии","[303, 302, 23, 36, 182]","[Саратовская область СШОР ""Надежда Губернии, У...","[610, 609, 24, 37, 183]",True,True
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе...","[303, 36, 71, 193, 173]","[Саратовская область СШОР ""Надежда Губернии, Н...","[610, 37, 72, 194, 174]",True,True
4,609,"""СШ ""Гвоздика""","[302, 172, 180, 184, 160]","[Удмуртская республика ""СШ ""Гвоздика"", ХМАО-Юг...","[609, 173, 181, 185, 161]",True,True


70.39% точности определения индекса школы с первого раза, и 92.29% что он будет в пятерке предполагаемых результатов

сравним результаты дообучающейся модели с результатами модели обученной сразу на всем датасете

In [70]:
df_big

,name,name_orig,school_id,name_aug
0,Московская область Авангард,Московская область Авангард,1,Москлвская область Авангард
1,Ямало Ненецкий АО Авангард,Ямало-Ненецкий АО Авангард,2,фмало Ненецкий АО Авангард
2,Республика Татарстан Авиатор,Республика Татарстан Авиатор,3,Республика Татарытан Авиатор
3,Санкт Петербург Аврора,Санкт-Петербург Аврора,4,Санкт Петербург Асрора
4,Санкт Петербург Ice Dream Айс Дрим,Санкт-Петербург Ice Dream / Айс Дрим,5,Санкт Ретербург ыce Dream Айс Дрим
...,...,...,...,...
2137,Алтайский край Прогресс,Алтайский край Прогресс,305,лтайски край Прогресс
2138,Удмуртская республика СШ Гвоздика,"Удмуртская республика ""СШ ""Гвоздика""",609,Ууртская рсублика СШ Гвоздика
2139,Саратовская область СШОР Надежда Губернии,"Саратовская область СШОР ""Надежда Губернии",610,Саатовсая область СШОР Надежда убении
2140,Пермский край КФК Айсберг,Пермский край КФК «Айсберг»,611,Прмски край КФК Айерг


In [71]:
df_for_test2 = pd.read_csv('C:\\Users\\Freo\\Desktop\\kaggle\\protect\\Примерное написание.csv')

In [72]:
df_for_test2

,school_id,name
0,1836,"ООО ""Триумф"""
1,1836,"Москва, СК ""Триумф"""
2,610,"СШОР ""Надежда Губернии"
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
4,609,"""СШ ""Гвоздика"""
...,...,...
890,3,"Республика Татарстан, СШОР ФСО Авиатор"
891,3,"СШОР ФСО Авиатор, Республика Татарстан"
892,3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»"
893,2,"ЯНАО, СШ ""Авангард"""


In [73]:
model_alt = SentenceTransformer('sentence-transformers/LaBSE')

corpus = model_alt.encode(df['name'].values)
query = model_alt.encode(x_test_final.values)
search_result = semantic_search(query,corpus, top_k=5)

df_for_test2['candidate_indxs'] = [[x['corpus_id'] for x in row] for row in search_result]
df_for_test2['candidate_names'] = df_for_test2['candidate_indxs'].transform(lambda x: df['name_orig'].values[x])
df_for_test2['candidate_school_id'] = df_for_test2['candidate_indxs'].transform(lambda x: df['school_id'].values[x])
df_for_test2['top1'] = df_for_test2.apply(lambda x: x['school_id'] == x['candidate_school_id'][0], axis=1)
df_for_test2['top5'] = df_for_test2.apply(lambda x: x['school_id'] in x['candidate_school_id'], axis=1)

In [74]:
df_for_test2['top1'].sum() / df_for_test2.shape[0]

0.6960893854748603

In [75]:
df_for_test2['top5'].sum() / df_for_test2.shape[0]

0.9195530726256983

Дообученная модель показала лучшие результаты чем обученная с нуля

## Выводы и итоги работы

В ходе работы были загружены и изучены два датасета с данными, эталонный с правильными заполнениями и случаные примеры.
В эталонном датасете имеются небольшие ошибки в названиях, но это не критично, в ходе работы моделируются данные с разными возможными ошибками для его расширения.

* проанализирована бейслайн модель на стартовом датасете
* было созданно несколько аугментированных обучающих датасетов под разные модели машинного обучения, содержащих в себе типовые ошибки:
    * пропущенные буквы
    * соседние буквы на клавиатуре (опечатки)
    * пары букв меняющихся местами
* подготовлена модель sentence-transformers/LaBSE для сопоставления произвольных и эталонных вариантов написания названий
* модель дообучена
* выполнено тестирование, сравнены результаты иходной и дообученой модели
* в процессе работы подробно описан ход выполнения
* написаны выводы и рекомендации

результаты и метрики:

| этап | модель | гиперпараметры | aссuracy предсказания | aссuracy (будет в пятерке)|
|----------|----------|----------|----------|----------|
| baseline   | линейная регрессия | 'liblinear'  | 0.8413   | -   |
| первый моделированный датасет   | линейная регрессия   | 'liblinear', penalty='l2', C=20, random_state=RSTATE, class_weight = 'balanced   | 0.7675   | -   |
| второй моделированный датасет   | sentence-transformers/LaBSE   | -   | 0.6960   | 0.9195   |
| после обучения на 20% данных   | sentence-transformers/LaBSE   | -   | 0.1932   | 0.2002   |
| после дообучения   | sentence-transformers/LaBSE   | -  | 0.6983   | 0.9217   |

идеи и улучшения:
* выделить отдельными признаками (с удалением из названия) принадлежность к типу "город" "область" "республика" итп в числовые признаки формата аналогичного OHE (1-0)
* попробовать решать проблему с обратной стороны - попробовать найти и распарсить базу всех существующих учебных учереждений, и подготовить для них аугемнтированную базу ошибок при заполнении
* попробовать увеличить датасет аугментированных ошибок и "донастроить" их